This is a notebook version of a lab/stencil file from the textbook "Coding the Matrix"

These files should NOT be made public, both to avoid sharing solutions, and to allow the book to maintain its value.
 
This stencil for the lab was originally created by Philip Klien, who holds the copyright.

This is a derivative work (converting to a notebook), designed for classroom simplicity, not as a separate product.

# Lab: Comparing Voting Records Using Dot-Products

### Importing the necessary data

In [101]:
import numpy as np

# uncomment to print entire NP matrix
# np.set_printoptions(threshold=np.nan)

# Requirements: the data file 'voting_record_dump109.txt' is included in the same folder as the notebook
f = open('../Datasets/US_Senate_voting_data_109.txt')
mylist = list(f)

### Task 1: Create Voting Dictionary

In [102]:
def parse_senator(senator, party_designation=None, verbose=False):    
    s_array = senator.split()
    
    #ints of the character votes
    int_votes = list(map(int,s_array[3:]))
    
    #keep name and voting history
    if verbose:
        s_key = (s_array[0], s_array[2])
    else:
        s_key = s_array[0]
    s_voting_dict_elmt = (s_key, int_votes)
    if party_designation:
        if s_array[1] == party_designation:
            return s_voting_dict_elmt
        #not a party we care about
        else:
            return (None,None)
    else:
        return s_voting_dict_elmt
        
        
def create_voting_dict(strlist, party_flag=None):
    """
    Input: a list of strings.  Each string represents the voting record of a senator.
           The string consists of 
              - the senator's last name, 
              - a letter indicating the senator's party,
              - a couple of letters indicating the senator's home state, and
              - a sequence of numbers (0's, 1's, and negative 1's) indicating the senator's
                votes on bills
              all separated by spaces.
              
    Output: A dictionary that maps the last name of a senator
            to a list of numbers representing the senator's voting record.
    Example: 
        >>> vd = create_voting_dict(['Kennedy D MA -1 -1 1 1', 'Snowe R ME 1 1 1 1'])
        >>> vd == {'Snowe': [1, 1, 1, 1], 'Kennedy': [-1, -1, 1, 1]}
        True

    You can use the .split() method to split each string in the
    strlist into a list; the first element of the list will be the senator's
    name, the second will be his/her party affiliation (R or D), the
    third will be his/her home state, and the remaining elements of
    the list will be that senator's voting record on a collection of bills.

    You can use the built-in procedure int() to convert a string
    representation of an integer (e.g. '1') to the actual integer
    (e.g. 1).

    The lists for each senator should preserve the order listed in voting data.
    In case you're feeling clever, this can be done in one line.
    """
    voting_dict = dict(map(lambda sen_str : parse_senator(*[sen_str, party_flag]), strlist))
    
    voting_dict.pop(None, None)
    
    return voting_dict
    
    

### Test Code for Task #1
The following code can be used to test your 'create_voting_dict' procedure. It should be removed before exporting to the script file for auto-grading.

In [103]:
vd = create_voting_dict(['Kennedy D MA -1 -1 1 1', 'Snowe R ME 1 1 1 1'])
print(vd)
vd == {'Snowe': [1, 1, 1, 1], 'Kennedy': [-1, -1, 1, 1]}

{'Kennedy': [-1, -1, 1, 1], 'Snowe': [1, 1, 1, 1]}


True

### Task 2: Write a policy compare procedure

In [104]:
import numpy as np

def policy_compare(sen_a, sen_b, voting_dict):
    """
    Input: last names of sen_a and sen_b, and a voting dictionary mapping senator
           names to lists representing their voting records.
    Output: the dot-product (as a number) representing the degree of similarity
            between two senators' voting policies
    Example:
        >>> voting_dict = {'Fox-Epstein':[-1,-1,-1,1],'Ravella':[1,1,1,1]}
        >>> policy_compare('Fox-Epstein','Ravella', voting_dict)
        -2
    
    The code should correctly compute the dot-product even if the numbers are not all in {0,1,-1}.
        >>> policy_compare('A', 'B', {'A':[100,10,1], 'B':[2,5,3]})
        253
        
    You should definitely try to write this in one line.
    """
    return np.dot(voting_dict[sen_a], voting_dict[sen_b])
    

### Test code for Task #2
The following code can be used to test 'policy_compare'

In [105]:
voting_dict = {'Fox-Epstein':[-1,-1,-1,1],'Ravella':[1,1,1,1]}
print(policy_compare('Fox-Epstein','Ravella', voting_dict))
print(policy_compare('A', 'B', {'A':[100,10,1], 'B':[2,5,3]}))
print(policy_compare('A', 'B', {'A':[1,1,1], 'B':[1,1,-1]}))

-2
253
1


### Task 3: Most Similar Procedure

In [106]:
def most_similar(sen, voting_dict):
    """
    Input: the last name of a senator, and a dictionary mapping senator names
           to lists representing their voting records.
    Output: the last name of the senator whose political mindset is most
            like the input senator (excluding, of course, the input senator
            him/herself). Resolve ties arbitrarily.
    Example:
        >>> vd = {'Klein': [1,1,1], 'Fox-Epstein': [1,-1,0], 'Ravella': [-1,0,0]}
        >>> most_similar('Klein', vd)
        'Fox-Epstein'
        >>> vd == {'Klein': [1,1,1], 'Fox-Epstein': [1,-1,0], 'Ravella': [-1,0,0]}
        True
        >>> vd = {'a': [1,1,1,0], 'b': [1,-1,0,0], 'c': [-1,0,0,0], 'd': [-1,0,0,1], 'e': [1, 0, 0,0]}
        >>> most_similar('c', vd)
        'd'

    Note that you can (and are encouraged to) re-use your policy_compare procedure.
    """
    # copy dict but remove exisint Senator that we are comparing
    # helpful for more consice iteration below
    others = voting_dict.copy()
    others.pop(sen)
    sen_names = others.keys()

    # start with the first Senator in the dict without our given Senator
    # maintain intermmediate twin with most comparable voting history
    twin_name = others.popitem()[0]
    twin_score = policy_compare(sen, twin_name, voting_dict)
        
    # iterate through senators and update the twin upon discovery of a more matched voter.
    for name in sen_names:
        voting_delta = policy_compare(sen, name, voting_dict)
        if (voting_delta > twin_score):
            twin_name = name
            twin_score = voting_delta
        
    return twin_name

Testing Code

In [107]:
vd = {'Klein': [1,1,1], 'Fox-Epstein': [1,-1,0], 'Ravella': [-1,0,0]}
print(most_similar('Klein', vd))  #Expect output to be 'Fox-Epstein'
print(vd == {'Klein': [1,1,1], 'Fox-Epstein': [1,-1,0], 'Ravella': [-1,0,0]} )  #Expected output, 'True'

vd = {'a': [1,1,1,0], 'b': [1,-1,0,0], 'c': [-1,0,0,0], 'd': [-1,0,0,1], 'e': [1, 0, 0,0]}
print(most_similar('c', vd)) #Expected output, 'd'

Fox-Epstein
True
d


### Task 4: Least Similar Procedure

In [108]:
def least_similar(sen, voting_dict, score_required=False):
    """
    Input: the last name of a senator, and a dictionary mapping senator names
           to lists representing their voting records.
    Output: the last name of the senator whose political mindset is least like the input
            senator.
    Example:
        >>> vd = {'a': [1,1,1], 'b': [1,-1,0], 'c': [-1,0,0]}
        >>> least_similar('a', vd)
        'c'
        >>> vd == {'a': [1,1,1], 'b': [1,-1,0], 'c': [-1,0,0]}
        True
        >>> vd = {'a': [-1,0,0], 'b': [1,0,0], 'c': [-1,1,0], 'd': [-1,1,1]}
        >>> least_similar('c', vd)
        'b'
    """
    # copy dict but remove exisint Senator that we are comparing
    # helpful for more consice iteration below
    others = voting_dict.copy()
    others.pop(sen)
    sen_names = others.keys()

    # start with the first Senator in the dict without our given Senator
    # maintain intermmediate twin with most comparable voting history
    twin_name = others.popitem()[0]
    twin_score = policy_compare(sen, twin_name, voting_dict)
        
    # iterate through senators and update the twin upon discovery of a more matched voter.
    for name in sen_names:
        voting_delta = policy_compare(sen, name, voting_dict)
        if (voting_delta < twin_score):
            twin_name = name
            twin_score = voting_delta
        
    return (twin_name, twin_score) if score_required else twin_name


In [109]:
vd = {'a': [1,1,1], 'b': [1,-1,0], 'c': [-1,0,0]}
print(least_similar('a', vd)) #expected output 'c'
print( vd == {'a': [1,1,1], 'b': [1,-1,0], 'c': [-1,0,0]} ) #Expected output 'True'

vd = {'a': [-1,0,0], 'b': [1,0,0], 'c': [-1,1,0], 'd': [-1,1,1]}
print(least_similar('c', vd)) #Expected output 'b'

c
True
b


### Task 5: Senators Chafee and Santorum

In [110]:
most_like_chafee = most_similar('Chafee', create_voting_dict(mylist))
print(most_like_chafee)

least_like_santorum = least_similar('Santorum', create_voting_dict(mylist))
print(least_like_santorum)

Jeffords
Feingold


### Task 6: Most Average Democrat

In [111]:
import numpy as np

def find_average_similarity(sen, sen_set, voting_dict):
    """
    Input: the name of a senator, a set of senator names, and a voting dictionary.
    Output: the average dot-product between sen and those in sen_set.
    Example:
        >>> vd = {'Klein':[1,1,1], 'Fox-Epstein':[1,-1,0], 'Ravella':[-1,0,0], 'Oyakawa':[-1,-1,-1], 'Loery':[0,1,1]}
        >>> sens = {'Fox-Epstein','Ravella','Oyakawa','Loery'}
        >>> find_average_similarity('Klein', sens, vd)
        -0.5
        >>> sens == {'Fox-Epstein','Ravella', 'Oyakawa', 'Loery'}
        True
        >>> vd == {'Klein':[1,1,1], 'Fox-Epstein':[1,-1,0], 'Ravella':[-1,0,0], 'Oyakawa':[-1,-1,-1], 'Loery':[0,1,1]}
        True
    """
    n = len(sen_set)
    v_sum = 0.0
    
    for s in sen_set:
        v_sum += np.dot(voting_dict[sen], voting_dict[s])
        
    return v_sum/n
        


In [112]:
#Test Code
vd = {'Klein':[1,1,1], 'Fox-Epstein':[1,-1,0], 'Ravella':[-1,0,0], 'Oyakawa':[-1,-1,-1], 'Loery':[0,1,1]}
sens = {'Fox-Epstein','Ravella','Oyakawa','Loery'}

print(find_average_similarity('Klein', sens, vd) ) 
# Expected output :   -0.5

print(sens == {'Fox-Epstein','Ravella', 'Oyakawa', 'Loery'} ) 
#Expected output:   True

print(vd == {'Klein':[1,1,1], 'Fox-Epstein':[1,-1,0], 'Ravella':[-1,0,0], 'Oyakawa':[-1,-1,-1], 'Loery':[0,1,1]} ) 
#expected output:   True\

-0.5
True
True


In [113]:
# give the last name (or code that computes the last name)
dem_dict = create_voting_dict(mylist, 'D')
dem_sen_names = set(dem_dict.keys())
name0 = dem_sen_names.pop()
score0 = find_average_similarity(name0, dem_sen_names, dem_dict)
most_avg_dem = (name0, score0)

for dem in dem_sen_names:
    score = find_average_similarity(dem, dem_sen_names, dem_dict)
    if score > most_avg_dem[1]:
        most_avg_dem = (dem, score)

print(most_avg_dem)

('Biden', 34.69047619047619)


### Task 7: Average Record

In [114]:
def find_average_record(sen_set, voting_dict):
    """
    Input: a set of last names, a voting dictionary
    Output: a vector containing the average components of the voting records
            of the senators in the input set
    Example: 
        >>> voting_dict = {'Klein': [-1,0,1], 'Fox-Epstein': [-1,-1,-1], 'Ravella': [0,0,1]}
        >>> senators = {'Fox-Epstein','Ravella'}
        >>> find_average_record(senators, voting_dict)
        [-0.5, -0.5, 0.0]
        >>> voting_dict == {'Klein': [-1,0,1], 'Fox-Epstein': [-1,-1,-1], 'Ravella': [0,0,1]}
        True
        >>> senators
        {'Fox-Epstein','Ravella'}
        >>> d = {'c': [-1,-1,0], 'b': [0,1,1], 'a': [0,1,1], 'e': [-1,-1,1], 'd': [-1,1,1]}
        >>> find_average_record({'a','c','e'}, d)
        [-0.6666666666666666, -0.3333333333333333, 0.6666666666666666]
        >>> find_average_record({'a','c','e','b'}, d)
        [-0.5, 0.0, 0.75]
        >>> find_average_record({'a'}, d)
        [0.0, 1.0, 1.0]
    """        
    
    voting_list_set = [voting_dict.get(sen) for sen in sen_set]
    n = len(sen_set) * 1.0  

    return np.sum(voting_list_set, axis=0) / n

In [115]:
#Test your procedure here...
#Use the previous tasks descriptions, and their test code to determine what you should extract from the above definition cell.
# d = {'c': [-1,-1,0], 'b': [0,1,1], 'a': [0,1,1], 'e': [-1,-1,1], 'd': [-1,1,1]}
# find_average_record({'a','c','e'}, d)

voting_dict = {'Klein': [-1,0,1], 'Fox-Epstein': [-1,-1,-1], 'Ravella': [0,0,1]}
senators = {'Fox-Epstein','Ravella'}
print(find_average_record(senators, voting_dict))

[-0.5 -0.5  0. ]


In [116]:
dem_dict = create_voting_dict(mylist, 'D')
avg_dem_rec = find_average_record(dem_dict.keys(), dem_dict)
print(avg_dem_rec)

[-0.1627907  -0.23255814  1.          0.8372093   0.97674419 -0.13953488
 -0.95348837  0.81395349  0.97674419  0.97674419  0.90697674  0.76744186
  0.6744186   0.97674419 -0.51162791  0.93023256  0.95348837  0.97674419
 -0.39534884  0.97674419  1.          1.          1.          0.95348837
 -0.48837209  1.         -0.3255814  -0.06976744  0.97674419  0.86046512
  0.97674419  0.97674419  1.          1.          0.97674419 -0.34883721
  0.97674419 -0.48837209  0.23255814  0.88372093  0.44186047  0.90697674
 -0.90697674  1.          0.90697674 -0.30232558]


### Task 8: Bitter Rivals

In [117]:
def bitter_rivals(voting_dict):
    """
    Input: a dictionary mapping senator names to lists representing
           their voting records
    Output: a tuple containing the two senators who most strongly
            disagree with one another.
    Example: 
        >>> voting_dict = {'Klein':[-1,0,1], 'Fox-Epstein':[-1,-1,-1], 'Ravella':[0,0,1], 'Oyakawa':[1,1,1], 'Loery':[1,1,0]}
        >>> br = bitter_rivals(voting_dict)
        >>> br == ('Fox-Epstein', 'Oyakawa') or br == ('Oyakawa', 'Fox-Epstein')
        True
    """
    
    # LinAlg implementation
    
    # data fetch
    rival_name_list = list(voting_dict.keys())
    voting_matrix = np.matrix(list(voting_dict.values()))  
        
    # compute a matrix lookup for senator comparison ratings
    rival_matrix = np.dot(voting_matrix, np.transpose(voting_matrix))
        
    # find the index of the minimals comparison ratings
    # , which corresponds to the 2 most differing senators
    rival_indices_proxy = np.where(rival_matrix == rival_matrix.min())
    rival_indices = next(zip(*rival_indices_proxy))\
    
    rivals = rival_name_list[rival_indices[0]], rival_name_list[rival_indices[1]]
    return rivals
    

    # quick python implementation:
        
#     sen_rival_scores = [(sen, least_similar(sen, voting_dict, True)) for sen in voting_dict.keys()]        
#     bitter_rivals = min(sen_rival_scores, key=lambda tup: tup[1][1])
#     return (bitter_rivals[0], bitter_rivals[1][0])
    


In [118]:
#Again, test your 'bitter_rivals' procedure here!
voting_dict = {'Klein':[-1,0,1], 'Fox-Epstein':[-1,-1,-1], 'Ravella':[0,0,1], 'Oyakawa':[1,1,1], 'Loery':[1,1,0]}
br = bitter_rivals(voting_dict)
br == ('Fox-Epstein', 'Oyakawa') or br == ('Oyakawa', 'Fox-Epstein')

True

### Task 9: Open Ended-Study
Create a separate notebook file that answers the open-ended study questions posed in the lab handout.
You should turn in BOTH the code, and the explanations of your answers (i.e. more than just 'yes', 'no', and names!)

  1. Who is most R/D
    1. Senator? 
    2. State?
  2. Is McCain a Maverick?
  3. Is Obama an Extremist?
  4. Who has most opponents (based on some threshold)?

#### Analysis: Python Prep

In [119]:
# allow for voting dictionary construct to 
# first seperate on Party affiliation
def create_verbose_voting_dict(strlist, party_flag=None):
    voting_dict = dict(map(lambda sen_str : parse_senator(*[sen_str, party_flag, True]), strlist))
    voting_dict.pop(None, None)
    return voting_dict

# for cleaner code analysis later on
def print_div():
    print("--------------\n")
       
# prints the Senators ID alongside their 
# similarity to the given voting dict
def print_senator_with_score(sen_key, voting_dict):
    print(sen_key[0] + " Score: " + str(find_average_similarity(sen_key, voting_dict.keys(), voting_dict)))

repubs = create_verbose_voting_dict(mylist, 'R')
dems = create_verbose_voting_dict(mylist, 'D')

# entire Senate (assuming 2 parties)
voting_dict = dems.copy()
voting_dict.update(repubs.copy())

#### Derivation of the most Republican/Democratic senators
When considering which Senator illustrates the quintessential (a) Republican or (b) Democrat, we must first establish a basis for qauntifying the average party  affiliate.  We do so by deriving the average voting record for the party, then determining which Senator amongst the party has the most comparable voting record to the parties established average:

In [120]:
def averge_party_member(party_dict):
    """
    Input: a dictionary mapping senator from a target political
           party names to lists representing their voting records.
    Output: ID of the senator of the which is most similar to the
            average amongst the target party.
    """
    
    # dummy senator ID to reresent the party's average voting record
    key = ('party', 'USA')
    
    # persist to dict
    avg_rec = find_average_record(party_dict.keys(), party_dict)
    dict_proxy = party_dict.copy()
    dict_proxy.update([(key, avg_rec)])
    
    # find Senator whose voting record is most similar 
    # to the party's average voting record identified
    return most_similar(key, dict_proxy)
 
avg_dem = averge_party_member(dems)
print("Average Dem: ")

# display avg dem relative to all of Senate
print_senator_with_score(avg_dem, voting_dict)

print_div()

avg_rep = averge_party_member(repubs)
print("Average Rep: ")

# display avg repub relative to all of Senate
print_senator_with_score(avg_rep, voting_dict)

Average Dem: 
Biden Score: 26.051020408163264
--------------

Average Rep: 
Allen Score: 31.459183673469386


Given the results above we can reasonably conclude that Biden illustrates the most _Democratic_ senator, and Allen is the best representative for the most _Republican_ senator.

#### Derivation of the most Republican/Democratic states
In a similar manner, if we group the dictionary to lookup via state abbreviations as opposed to senator names, then simply lookup which state's representative voting record is most similar to the most average record for each party, we can determine which states most appropriately represent _Republican_-ness and _Democratic_-ness.

In [121]:
# constructing a dictionary of states to their 
# corresponding average voting record
states = dict()
for sen in voting_dict.keys():
    state_name = sen[1]
    votes = voting_dict[sen]
    
    # appends an additional senators
    # voting record to this present state.
    # to later be averaged for representing
    # the specific states "vote" 
    if state_name in states:
        state = states[state_name]
        state.append(votes)
    else:
        states[state_name] = [votes]

# averaging the votes
states = {k: np.mean(v, axis=0) for k, v in states.items()}


def averge_state_helper(state_dict, avg_party_member):
    """
    Input: a dictionary mapping states from a target political
           party names to lists representing their voting records.
           , average senator from target party
    Output: tuple with the ID of the state of the which is most similar to the
            average amongst the target party, and the states agreeance score 
            relative to the target party
    """
    # adds the average as "state" to the voting dict copy 
    dict_proxy = state_dict.copy()
    dict_proxy.update([avg_party_member])
    target_state = most_similar(avg_party_member[0], dict_proxy)
    
    # perist the state's voting score relative to all other states
    states_score = find_average_similarity(target_state, state_dict.keys(), state_dict)
    return (target_state, states_score)

print("Most Dem State: " + str(averge_state_helper(states, (avg_dem, dems[avg_dem]))))
print("Most Rep State: " + str(averge_state_helper(states, (avg_rep, repubs[avg_rep]))))

Most Dem State: ('MD', 23.82)
Most Rep State: ('MO', 31.09)


Given the results above we can reasonably conclude that Maryland (MD) illustrates the most _Democratic_ state, while Missouri (MO) is the best representative for the most _Republican_ leaning state.  When we compare the similarities between the scores of MD (23.82) and Biden (26.051), to that of MO (31.09) and Allen (31.459) we can further reinforce the aforementioned conclusions.

#### Is Obama an Extremist?  
In order to determine if Obama is deserving of such a title, we must determine his position relative to the Republican Party.  This first requires us to determine which Democratic senators represent the largest gap in agreeance amongst all Democrats, also known as the most bitter Democratic rivals. Secondly, we must then compute and analyze how each of those three senators compare with their rival party:

In [122]:
d_rivals = bitter_rivals(dems)
rep_plus_rival = repubs.copy()
obama = ('Obama', 'IL')

print("Extreme Dems: ", d_rivals)

for sen in [obama, *d_rivals]:
    # target senator must be present in dict to find score 
    rep_plus_rival.update([(sen, voting_dict[sen])])
    print_senator_with_score(sen, rep_plus_rival)
    # remove the singular opponent from the party
    rep_plus_rival.pop(sen)

Extreme Dems:  (('Feingold', 'WI'), ('Nelson2', 'NE'))
Obama Score: 21.071428571428573
Feingold Score: 5.678571428571429
Nelson2 Score: 38.107142857142854


As discovered above, the most differing Democrats are Senators Feingold and Nelson2.  In analyzing how they compare relative to Obama from the perspective of the Republican Party,  it's quite apparent that Feingold presents a much greater opposing voting record than Obama does.  In fact, Obama appears to represent the stereortypical Democratic from the Republicans purview given his agreeance score with them bisects that of rivaling Democrats Feingold and Nelson2 (who arguably is more Republican than Democratic in vote).

#### Is McCain an Eccentric?
In order to determine if Obama is deserving of such a title, we must determine his position relative to the Democratic Party.  This first requires us to determine which Republican senators represent the largest gap in agreeance amongst all Democrats, also known as the most bitter Republican rivals. Secondly, we must then compute and analyze how each of those three senators compare with their rival party:

In [123]:
r_rivals = bitter_rivals(repubs)
dem_plus_rival = dems.copy()
mccain = ('McCain', 'AZ')

print("Extreme Reps: ", r_rivals)

for sen in [mccain, *r_rivals]:
    # target senator must be present in dict to find score 
    dem_plus_rival.update([(sen, voting_dict[sen])])
    print_senator_with_score(sen, dem_plus_rival)
    # remove the singular opponent from the party
    dem_plus_rival.pop(sen)

Extreme Reps:  (('Chafee', 'RI'), ('Coburn', 'OK'))
McCain Score: 14.704545454545455
Chafee Score: 29.204545454545453
Coburn Score: 11.727272727272727


As discovered above, the most differing Republican senators are Chafee and Coburn. In analyzing how they compare relative to McCain from the perspective of the Democratic Party we can conclude that McCain presents an extreme much more comparable to Cobourn (Democrats most extreme opponen) than Chafee does, whom represents a much more modest dissent.  Thus we can conclude that McCain can be reasonably described as a maverick of Republican ideals from the perspective of opposition.

#### Extremist Sprectrum
When determining the number of political opponents based on some pre-defined threshold, one may first come upon the simple metric of comparing the minimal agreed votes between senators to gauge their rivalry, which we can easily compute by taking the dot product of the Senate's voting matrix, along with it's own transpose.  

In [124]:
def opponents(threshold, voting_dict):
    """
    Input: comparing threshold (represents minimal similar votes)
           , voting dictionary
    Output: tuple with the ID of the senator of the which has 
            the most number of senators whom ball below the specified
            threshold, and the number aforementioned senators
    """
    ordered_sen_names = list(voting_dict.keys())
    most_hated = (ordered_sen_names[0], 0)
    voting_matrix = np.matrix(list(voting_dict.values())) 
    rival_matrix = np.dot(voting_matrix, np.transpose(voting_matrix))

    for name, sen in zip(ordered_sen_names, rival_matrix):
        opps = np.where(sen < threshold)
        n_opps = len(opps[0])
        if n_opps > most_hated[1]:
            most_hated = (name, n_opps)
    return most_hated

for i in range(0, 50, 5):
    print("Most rivaled at least", i, "agreed votes:", opponents(i, voting_dict))

Most rivaled at least 0 agreed votes: (('Feingold', 'WI'), 4)
Most rivaled at least 5 agreed votes: (('Feingold', 'WI'), 29)
Most rivaled at least 10 agreed votes: (('Feingold', 'WI'), 51)
Most rivaled at least 15 agreed votes: (('Feingold', 'WI'), 57)
Most rivaled at least 20 agreed votes: (('Feingold', 'WI'), 62)
Most rivaled at least 25 agreed votes: (('Feingold', 'WI'), 72)
Most rivaled at least 30 agreed votes: (('Coburn', 'OK'), 94)
Most rivaled at least 35 agreed votes: (('Lieberman', 'CT'), 97)
Most rivaled at least 40 agreed votes: (('Lieberman', 'CT'), 98)
Most rivaled at least 45 agreed votes: (('Akaka', 'HI'), 98)


In the above snippet, we're logging the particular senator who has the greatest number of senators to have agreed with them no less than X times where X is in the range of increments of 5 from \[0 to 50\).  Surprisingly, Senator Feingold has actively disagreed with four senators more than they've voted in agreeance.  Feingold maintains this title through having agreed with the greatest number of senators on no less than 25 occasions.  Given the 98 votes sampled, we can determine that such a threshold (25 minimal agreeances) correspondes to an overall agreeance range of:
  ```math
  25.5% (= (25 agreements + 73 pass) / 98 votes) to
  62.2% (= (61 agreements + 36 disagreements + 1 pass) / 98 votes) 
  ```
  
From the specified number of opponents which satisfy thris threshold for Feingold, 72, it's safe to conclude that, within the scope of these samples votes, Feingold has ruffled the feathers of nearly 72\% the Senate (their party inclusive!).  However, right up Feingold's nose at the 31\% - 66\% agreeance range is Senator Coburn.  Coburn is also considered to be the most extremist of the Republicans.  With such a slight range shift causing such a drastic increase in the total number of enemies (94 senators), it's moreso likely Coburn has stepped on more toes than Feingold.  Given our defined threshold above, it's safe to conclude that Senator Coburn has the largest volume of political opponents (which is guaranteed to include a large portion of their very own party).
